In [2]:
!pip install opendatasets

In [3]:
import pandas as pd
import opendatasets as od

In [7]:
od.download("https://www.kaggle.com/datasets/bensalem14/sgcc-dataset")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mukiibirogers
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/bensalem14/sgcc-dataset


100%|██████████| 51.2M/51.2M [00:00<00:00, 87.9MB/s]
